# Supervised Learning Project

**by Octavio Garcia**

In [1]:
#Libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [4]:
bat = pd.read_csv('./Batting.csv')#,encoding = "ISO-8859-1")

In [5]:
display(bat.shape,bat.dtypes)

(105861, 22)

playerID     object
yearID        int64
stint         int64
teamID       object
lgID         object
G             int64
AB            int64
R             int64
H             int64
2B            int64
3B            int64
HR            int64
RBI         float64
SB          float64
CS          float64
BB            int64
SO          float64
IBB         float64
HBP         float64
SH          float64
SF          float64
GIDP        float64
dtype: object

In [6]:
bat.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [7]:
#df['Element'].unique()
#df['Item'].value_counts()
feats = bat.columns
feats

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [17]:
object_col = np.array(['teamID','lgID'])

In [46]:
def selected_feat(selected,df):
    values = []
    leng = selected.shape
    for i in range(leng[0]):
        values.append(df[selected[i]].unique())
        new_df = pd.DataFrame(values).T
    new_df.columns = [i for i in selected]
    return new_df

In [48]:
selected_feat(object_col,bat)

,teamID,lgID
0,TRO,NaN
1,RC1,NL
2,CL1,AA
3,WS3,UA
4,FW1,PL
5,BS1,AL
6,PH1,FL
7,CH1,None
8,NY2,None
9,MID,None


In [50]:
#Remove registers with null
bat.dropna(inplace=True)
bat.shape

(69181, 22)

In [57]:
bat.isna().sum()

playerID    0
yearID      0
stint       0
teamID      0
lgID        0
G           0
AB          0
R           0
H           0
2B          0
3B          0
HR          0
RBI         0
SB          0
CS          0
BB          0
SO          0
IBB         0
HBP         0
SH          0
SF          0
GIDP        0
dtype: int64

In [59]:
selected_feat(object_col,bat).head()

,teamID,lgID
0,BRO,NL
1,SLN,AL
2,PIT,None
3,CHN,None
4,PHI,None


In [62]:
bat.teamID.value_counts()

CLE    2777
CHN    2741
NYA    2719
SLN    2710
CHA    2683
PIT    2682
BOS    2677
PHI    2675
CIN    2644
DET    2637
BAL    2635
LAN    2505
NYN    2446
SFN    2442
HOU    2321
MIN    2274
ATL    2218
OAK    2204
SDN    2166
KCA    2111
TEX    2089
SEA    1839
TOR    1807
MON    1479
CAL    1315
COL    1208
ML4    1060
ARI     974
TBA     964
MIL     947
FLO     849
LAA     832
WAS     670
KC1     572
WS2     430
ML1     408
MIA     341
ANA     337
WS1     290
BRO     161
NY1     156
SE1      53
SLA      49
BSN      44
PHA      40
Name: teamID, dtype: int64

In [63]:
bat.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
12630,aitchra01,1911,1,BRO,NL,1,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
12701,camniha01,1911,1,SLN,NL,2,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
12739,cottren01,1911,1,PIT,NL,1,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
12831,griffha01,1911,1,CHN,NL,1,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
13046,pucketr01,1911,1,PHI,NL,1,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
bat_1975 = bat[bat.yearID >= 1975]

In [72]:
bat_1975.teamID.value_counts()

TEX    1971
NYA    1963
CLE    1937
NYN    1933
SDN    1931
OAK    1925
BOS    1912
PIT    1905
LAN    1905
KCA    1892
BAL    1887
PHI    1887
CIN    1869
ATL    1869
CHN    1868
SFN    1846
SEA    1839
DET    1839
CHA    1831
SLN    1824
TOR    1807
HOU    1796
MIN    1771
MON    1252
COL    1208
ARI     974
TBA     964
MIL     947
CAL     900
ML4     866
FLO     849
WAS     670
LAA     665
MIA     341
ANA     337
Name: teamID, dtype: int64

In [20]:
food_drop = food.drop(['Area Abbreviation', 
                       'Area Code', 'Item Code',
                       'Element Code','Unit',
                       'latitude','longitude'],axis=1)
food_drop.head()

,Area,Item,Element,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,Afghanistan,Wheat and products,Food,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,Afghanistan,Rice (Milled Equivalent),Food,183.0,183.0,182.0,220.0,220.0,195.0,231.0,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,Afghanistan,Barley and products,Feed,76.0,76.0,76.0,76.0,76.0,75.0,71.0,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,Afghanistan,Barley and products,Food,237.0,237.0,237.0,238.0,238.0,237.0,225.0,...,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,Afghanistan,Maize and products,Feed,210.0,210.0,214.0,216.0,216.0,216.0,235.0,...,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [24]:
food_dropna['Item'].value_counts()

Milk - Excluding Butter         450
Eggs                            298
Cereals - Excluding Beer        291
Fish, Seafood                   283
Maize and products              278
Pelagic Fish                    275
Oilcrops                        259
Cereals, Other                  255
Starchy Roots                   255
Animal fats                     241
Oats                            227
Fish, Body Oil                  225
Barley and products             223
Wheat and products              218
Rice (Milled Equivalent)        212
Pulses                          210
Oilcrops, Other                 209
Vegetables, Other               206
Cassava and products            206
Pulses, Other and products      206
Vegetables                      206
Potatoes and products           201
Fruits - Excluding Wine         188
Marine Fish, Other              186
Roots, Other                    184
Soyabeans                       180
Demersal Fish                   174
Rye and products            

In [25]:
items = ['Milk - Excluding Butter','Eggs','Cereals - Excluding Beer','Fish, Seafood',
         'Maize and products']

In [26]:
food_ = food_dropna[food_dropna['Item'].isin(items)]